# **TradeCare: Data Collection Notebook**

## Objectives
* Fetch historical Bitcoin OHLCV (Open, High, Low, Close, Volume) data from a GitHub-hosted repository that provides automated daily updates.
* Verify data loaded correctly (basic checks)
* Understand data structure and characteristics
* Document data source and live data collection strategy

## Inputs
*  **Data Source:** GitHub Repository (automated updates)
*   **URL:** https://raw.githubusercontent.com/mouadja02/bitcoin-hourly-ohclv-dataset/main/btc-hourly-price_2015_2025.csv\n
*   **Asset:** BTC-USD
*   **Timeframe:** 1 Hour
*   **Period:** November 2014 - present

## Outputs
* DataFrame loaded in memory for exploration
* Data understanding documented
* Live data approach: No CSV files saved (subsequent notebooks fetch fresh) 

## Additional Comments
This GitHub dataset provides a **unique combination** rarely found in ML projects:

* **Fresh & Maintained:** Automated workflow fetches current data from CryptoCompare API daily and stores backups on GitHub. Repository contains Bitcoin hourly price data from 2015 to present with continuous updates
* **Simple**: Direct CSV access via single URL
* **Free**: No API keys or costs  
* **Reliable**: No rate limits or auth failures  
* **Transparent**: Git history shows every change  
* **Scalable**: Should work in production environments  

**Live Data Strategy:**
* This notebook fetches data from URL and explores it
* No CSV files are saved (live data approach)
* Each notebook in the pipeline will fetch fresh data as needed
* This ensures always up-to-date analysis and predictions
* Trade-off: Requires internet connection, slightly slower but more current

---

## Change Working Directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with `os.getcwd()`

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Fetch and Validate Data

## Import Validation Helper

In [ ]:
# Import centralized validation function
import sys
sys.path.append('.')  # Add project root to path

from src.raw_data_validation import fetch_and_validate_data, get_data_info

## Fetch Validated Data

This function automatically:
- Fetches data from GitHub URL
- Validates column structure and names
- Validates string data safety (no injection attempts)
- Validates price ranges
- Validates data completeness
- Validates timestamps

If any validation fails, the notebook stops with a clear error message.

In [ ]:
# Fetch and validate data in one call
df = fetch_and_validate_data()

---

# Inspect Data 

## Validate Data Integrity

## Load CSV Data

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.